In [1]:
import numpy as np
import os
import sys
import pandas as pd
import random
import yaml
import numpy as np
import random
import torch
import os
import warnings
warnings.filterwarnings('ignore')


def seed_everything(seed: int = 42):
    random.seed(seed)
    np.random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)  # type: ignore
    torch.backends.cudnn.deterministic = True  # type: ignore
    torch.backends.cudnn.benchmark = True  # type: ignore

seed_everything(777)

c:\Users\user\anaconda3\envs\minmin\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
!pip install pandas==0.25.3

In [6]:
bin_data = pd.read_csv(r'E:\Datasets\Network_intrusion/IoT_original.csv')
bin_data_select = bin_data.drop(['Flow_ID','Src_IP','Label','Cat','Dst_IP'], axis=1)
bin_data_1 = bin_data_select.copy()

In [7]:
# labeling 처리

from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
result = le.fit_transform(bin_data_1['Sub_Cat'])
print(result)
print(le.classes_)

bin_data_1['label'] = result

bin_data_1 = bin_data_1.drop(['Sub_Cat'], axis=1)


[2 0 8 ... 8 6 5]
['DoS-Synflooding' 'MITM ARP Spoofing' 'Mirai-Ackflooding'
 'Mirai-HTTP Flooding' 'Mirai-Hostbruteforceg' 'Mirai-UDP Flooding'
 'Normal' 'Scan Hostport' 'Scan Port OS']


In [8]:
## 전처리 진행
# 1) inf 값, na 값 제외
bin_data_2 = bin_data_1.replace(np.inf, np.nan)
bin_data_2 = bin_data_2.replace(-np.inf, np.nan)

bin_data_2 = bin_data_2.dropna()

In [9]:
# 0의 비중이 절반 이상인 컬럼 제외
z_df=pd.DataFrame((bin_data_2==0).sum() / bin_data_2.shape[0]).sort_values([0], ascending=False)
drop_list_zero=list(z_df.index[z_df[0]>0.5])
print(len(drop_list_zero))
bin_data_2.drop(columns=drop_list_zero, inplace=True)

39


In [6]:
df_full_data=bin_data_2.describe()

In [10]:
df_full_data=bin_data_2.describe()
df_full_data.columns[df_full_data.loc['std']==0]

Index(['Init_Fwd_Win_Byts'], dtype='object')

In [11]:
## std 0인 컬럼 제거
bin_data_2 = bin_data_2.drop(['Init_Fwd_Win_Byts'],axis=1)

In [12]:
# time index 설정 
# minmax scaling 후 time index feature 추가
netwokrtime=pd.to_datetime(bin_data_2['Timestamp'])

bin_data_2=bin_data_2.set_index(netwokrtime).sort_index()
bin_data_2=bin_data_2.drop(['Timestamp'], axis=1)

In [13]:
## minmax scaler 적용
from sklearn.preprocessing import StandardScaler, MinMaxScaler
sc_minmax = MinMaxScaler()
bin_data_2_sc=sc_minmax.fit_transform(bin_data_2.iloc[:,:-1])

In [14]:
## df 로 변환
bin_data_2_sc_df = pd.DataFrame(bin_data_2_sc, columns= bin_data_2.columns[:-1])
bin_data_2_sc_df.index = bin_data_2.index
bin_data_2_sc_df['label'] = bin_data_2['label']

In [31]:
import numpy as np
import holidays
import warnings
warnings.filterwarnings('ignore')
# holiday 생성 함수 
# holiday 인 경우 1 그렇지 않으면 0 
import sys

# 위 함수를 사용하여 holiday column 추가

def generate_cyclical_features(df, col_name, period, start_num=0):
    kwargs = {
        f'sin_{col_name}' : lambda x: np.sin(2*np.pi*(df[col_name]-start_num)/period),
        f'cos_{col_name}' : lambda x: np.cos(2*np.pi*(df[col_name]-start_num)/period)    
             }
    return df.assign(**kwargs).drop(columns=[col_name])

input_df = bin_data_2_sc_df.copy()
## indes 값이 날짜로만 구성되어 있는지 확인
# 날짜로만 구성되어 있다면 아래와 같이 진행
if all(isinstance(x, pd.Timestamp) for x in input_df.index):

    # 원본 데이터를 포함한, time index의 feautre 추가
    df_features = (
            input_df
            .assign(hour = input_df.index.hour) # 0~23
            .assign(day = input_df.index.day) # 1~31
            .assign(month = input_df.index.month) # 3 ~ 4
            .assign(day_of_week = input_df.index.dayofweek) # 0 ~6
            .assign(week_of_year = input_df.index.isocalendar().week) # 10 ~ 18 # Extract week from isocalendar()
            .assign(second = input_df.index.second)
            # .assign(is_weekend = main_df_11.index.weekday)
          )
    # 기존 평일, 주말 데이터 변환
    df_features['is_week']=np.where((df_features['day_of_week']==5)|(df_features['day_of_week']==6),1,0)

    '''
    시간의 주기성을 이용하여, fourier 변환 방식 적용
    '''
    df_features = generate_cyclical_features(df_features, 'hour', 
                df_features['hour'].unique().shape[0], df_features['hour'].unique().min())

    df_features = generate_cyclical_features(df_features, 'day', 
                df_features['day'].unique().shape[0], df_features['day'].unique().min())

    df_features = generate_cyclical_features(df_features, 'month', 
                df_features['month'].unique().shape[0], df_features['month'].unique().min())

    df_features = generate_cyclical_features(df_features, 'day_of_week', 
                df_features['day_of_week'].unique().shape[0], df_features['day_of_week'].unique().min())

    df_features = generate_cyclical_features(df_features, 'week_of_year', 
                df_features['week_of_year'].unique().shape[0], df_features['week_of_year'].unique().min())

    df_features = generate_cyclical_features(df_features, 'second', 
                df_features['second'].unique().shape[0], df_features['second'].unique().min())

    '''
    holiday 생성을 위한 함수 적용
    '''
    us_holidays = holidays.KR()
    def is_holiday(date):
        
        date = date.replace(hour = 0)
        return 1 if (date in us_holidays) else 0

    def add_holiday_col(df):
        return df.assign(is_holiday = df.index.to_series().apply(is_holiday))

    # holiday data 추가 후 is_week 와 병합
    df_features = add_holiday_col(df_features)
    df_features3  =df_features.reset_index()
    df_features3['is_week'][df_features3[df_features3['is_holiday']==1].index]=1

    # is_hoiday 제거
    df_features_using = df_features3.drop(['is_holiday'], axis=1)

### 불필요한 컬럼제거 및 라벨 컬럼 재배치 

In [32]:
del df_features_using['Timestamp']
del df_features_using['label']

bin_data_4 = bin_data_2.reset_index() # 새로운 인덱스를 적용 
df_features_using['label'] = bin_data_4['label'] # 라벨 추가 

### csv 저장 

In [ ]:
df_features_using.to_csv('network전처리.csv')